In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

print('blah')

blah


In [3]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(8, 6),
            nn.Softmax(),
            nn.Linear(6, 4),
            nn.Softmax(),
            nn.Linear(4, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=8, out_features=6, bias=True)
    (1): Softmax(dim=None)
    (2): Linear(in_features=6, out_features=4, bias=True)
    (3): Softmax(dim=None)
    (4): Linear(in_features=4, out_features=1, bias=True)
  )
)


In [8]:
X = torch.rand(1, 8, device=device)
logits = model(X)
print(f"All returns: {logits}")

pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

/Users/logan/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


All returns: tensor([[-0.2537]], device='mps:0', grad_fn=<LinearBackward0>)
Predicted class: tensor([0], device='mps:0')


/Users/logan/opt/anaconda3/lib/python3.9/site-packages/torch/_tensor_str.py:115: UserWarning: The operator 'aten::nonzero' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1670525473998/work/aten/src/ATen/mps/MPSFallback.mm:11.)
  nonzero_finite_vals = torch.masked_select(


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

play_data = pandas.read_table('TrainingQueue.tsv')


X = play_data.drop(columns=['MyPaddleVelocity','Unnamed: 9']).dropna()
y = play_data['MyPaddleVelocity'].dropna() 



In [12]:
class PongDataset(Dataset):
    def __init__(self, X, y):
        self.features = X.values
        self.labels = y.values

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]
        return feature, label

play_data = pd.read_table('TrainingQueue.tsv')
X = play_data.drop(columns=['MyPaddleVelocity', 'Unnamed: 9']).dropna()
y = play_data['MyPaddleVelocity'].dropna()

In [13]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create data loaders for the training and validation sets
train_dataset = PongDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = PongDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [30]:
# Define the loss function and optimizer
loss_fn = nn.MSELoss()
learning_rate = .001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model for the specified number of epochs
num_epochs = 10
for epoch in range(num_epochs):
    for batch in train_loader:
        # Move the batch of training data to the device
        X_batch, y_batch = batch
        X_batch = X_batch.float()
        y_batch = y_batch.float()
        X_batch, y_batch = X_batch.to(device).float(), y_batch.to(device).float()
        
        # Forward pass
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch.unsqueeze(1))
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Evaluate the model on the validation set
    with torch.no_grad():
        for batch in val_loader:
            # Move the batch of validation data to the device
            X_batch, y_batch = batch
            X_batch = X_batch.float()
            y_batch = y_batch.float()
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            # Forward pass
            y_pred = model(X_batch)
            val_loss = loss_fn(y_pred, y_batch.unsqueeze(1))
            
        # Print progress
        print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")



Epoch [0/10], Train Loss: 2.3125, Val Loss: 4.4787
Epoch [1/10], Train Loss: 0.7172, Val Loss: 4.4162
Epoch [2/10], Train Loss: 2.1200, Val Loss: 4.3517
Epoch [3/10], Train Loss: 1.1473, Val Loss: 4.2826
Epoch [4/10], Train Loss: 2.2071, Val Loss: 4.2222
Epoch [5/10], Train Loss: 1.0643, Val Loss: 4.1645
Epoch [6/10], Train Loss: 0.7139, Val Loss: 4.0909
Epoch [7/10], Train Loss: 2.2810, Val Loss: 4.0214
Epoch [8/10], Train Loss: 2.4666, Val Loss: 3.9658
Epoch [9/10], Train Loss: 1.1426, Val Loss: 3.9113


In [31]:
# Evaluate the trained model on the test set
model.eval()



test_loss = 0
correct = 0
with torch.no_grad():
    for batch in val_loader:
        # Move the batch of testing data to the device
        X_batch, y_batch = batch
        X_batch, y_batch = X_batch.to(device).float(), y_batch.to(device).float()

        # Forward pass
        y_pred = model(X_batch)

        # Compute loss
        test_loss += loss_fn(y_pred, y_batch).item()

        # Compute number of correct predictions
        correct += (y_pred.argmax(1) == y_batch.argmax(1)).type(torch.float).sum().item()

test_loss /= len(val_loader.dataset)
accuracy = correct / len(val_loader.dataset)

print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.2%}')


NameError: name 'test_loader' is not defined